In [21]:
!pip install pymystem3

!pip install pymorphy2

!pip install jsonlines

from pymystem3 import Mystem

import pymorphy2
from pymorphy2 import MorphAnalyzer

import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

import json

import jsonlines

import string

import collections
from collections import Counter

import re

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


We start with opening the text and converting it's type to string:

In [22]:
with open('capote.txt', 'r') as file:
    text_in = file.read().rstrip()

text1 = text_in.translate(str.maketrans('', '', string.punctuation))

pattern = r'[0-9]'
text = re.sub(pattern, '', text1)

First we lemmatize the text:

In [6]:
m = Mystem()
lemmas = m.lemmatize(text)

text_out = open('mystem_capote.txt','w')
text_out.write(''.join(lemmas))
text_out.close()

Installing mystem to /root/.local/bin/mystem from http://download.cdn.yandex.net/mystem/mystem-3.1-linux-64bit.tar.gz


KeyboardInterrupt: ignored

Then we tokenize it:

In [23]:
tokens = nltk.word_tokenize(text)

Now we analyze the resulting words:

In [24]:
morph = MorphAnalyzer()

i = 0
lines = []

while i < len(tokens):
    ana = morph.parse(tokens[i])
    current = ana[0]
    d = {'lemma': current.normal_form, 'word': current.word, 'pos': current.tag.POS}
    lines.append(d)
    i += 1

with jsonlines.open('pymorphy_capote.jsonl', mode='w') as writer:
    for d in lines:
        writer.write(d)

**1) What percentage constitutes each pos? (E.g., for the verb, the number of verbs divided by the total number of words)**

To answer this question we need to make a list with every pos and it's number of entrances:

In [62]:
list_of_pos = []

for d in lines:
    pos = d.get('pos')
    list_of_pos.append(pos)

#print(list_of_pos)
print(Counter(list_of_pos))
#print(len(Counter(list_of_pos)))

Streaming output truncated to the last 5000 lines.
<class 'pymorphy2.tagset._select_grammeme_from.__init__.<locals>.TypedGrammeme'>
<class 'pymorphy2.tagset._select_grammeme_from.__init__.<locals>.TypedGrammeme'>
<class 'pymorphy2.tagset._select_grammeme_from.__init__.<locals>.TypedGrammeme'>
<class 'pymorphy2.tagset._select_grammeme_from.__init__.<locals>.TypedGrammeme'>
<class 'pymorphy2.tagset._select_grammeme_from.__init__.<locals>.TypedGrammeme'>
<class 'pymorphy2.tagset._select_grammeme_from.__init__.<locals>.TypedGrammeme'>
<class 'pymorphy2.tagset._select_grammeme_from.__init__.<locals>.TypedGrammeme'>
<class 'pymorphy2.tagset._select_grammeme_from.__init__.<locals>.TypedGrammeme'>
<class 'pymorphy2.tagset._select_grammeme_from.__init__.<locals>.TypedGrammeme'>
<class 'pymorphy2.tagset._select_grammeme_from.__init__.<locals>.TypedGrammeme'>
<class 'pymorphy2.tagset._select_grammeme_from.__init__.<locals>.TypedGrammeme'>
<class 'pymorphy2.tagset._select_grammeme_from.__init__.<l

And here is the number of all words in the text:

In [17]:
print(len(lines))

22603


Now all we need is to print out the percentage for every pos:

In [64]:
y = [[x,list_of_pos.count(x)] for x in set(list_of_pos)]

for i in y:
    print(i[0], " = ", round(i[1]/len(lines), 3))

#i = 0
#while i < len(c):
#    print(Counter(list_of_pos)[i], '= ')

<class 'list'>
NUMR  =  0.006
<class 'list'>
None  =  0.004
<class 'list'>
VERB  =  0.15
<class 'list'>
CONJ  =  0.105
<class 'list'>
PREP  =  0.101
<class 'list'>
PRTS  =  0.004
<class 'list'>
INTJ  =  0.002
<class 'list'>
GRND  =  0.01
<class 'list'>
PRTF  =  0.006
<class 'list'>
ADJS  =  0.007
<class 'list'>
INFN  =  0.03
<class 'list'>
ADVB  =  0.063
<class 'list'>
COMP  =  0.005
<class 'list'>
PRCL  =  0.058
<class 'list'>
NPRO  =  0.127
<class 'list'>
PRED  =  0.006
<class 'list'>
ADJF  =  0.09
<class 'list'>
NOUN  =  0.226


The strange category 'None' represents all the numbers, foreign words, etc. -- I decided to keep them since they are a legit part of text (and either way there are not so many of these).

---

**2) Print out top-20 verbs and adverbs.**

To answer this question we need to make a list with of every verb and another list of every adverb (it's not clear whether I need to prin out a top-20 of verbs and adverbs or a top-20 of verbs and top-20 of adverbs so I decided to do to do separate tops) and then to sort out top-20:

In [85]:
from scipy.special import k0
list_of_verbs = []
list_of_adverbs = []

for d in lines:
    if d["pos"] == 'VERB':
        verb = d["lemma"]
        list_of_verbs.append(verb)

print(Counter(list_of_verbs).most_common(20))

for d in lines:
    if d["pos"] == 'ADVB':
        adverb = d["lemma"]
        list_of_adverbs.append(adverb)

print(Counter(list_of_adverbs).most_common(20))

[('быть', 293), ('сказать', 146), ('мочь', 106), ('знать', 69), ('говорить', 49), ('хотеть', 47), ('любить', 40), ('казаться', 39), ('думать', 36), ('стать', 33), ('голайтлить', 30), ('видеть', 27), ('спросить', 22), ('дать', 19), ('идти', 19), ('жить', 18), ('взять', 18), ('стоять', 17), ('увидеть', 17), ('стоить', 16)]
[('только', 72), ('ещё', 68), ('там', 47), ('уже', 40), ('потому', 36), ('потом', 35), ('где', 34), ('теперь', 33), ('очень', 30), ('пока', 30), ('здесь', 20), ('тут', 20), ('далее', 20), ('всегда', 19), ('совсем', 19), ('ничего', 17), ('снова', 16), ('тогда', 16), ('несколько', 15), ('вдруг', 15)]


Next task is as follows:
**Find top-25 bigrams and trigrams for your text (use nltk.bigrams), use only lemmas, get rid of the punctuation. Comment shortly on the results.**

In [90]:
with open('mystem_capote.txt', 'r') as file:
    text_new = file.read().rstrip()

bigrm = list(nltk.bigrams(text_new.split()))
print(Counter(bigrm).most_common(25))

trigrm = list(nltk.trigrams(text_new.split()))
print(Counter(trigrm).most_common(25))

[(('я', 'не'), 63), (('что', 'я'), 53), (('сказать', 'она'), 51), (('у', 'я'), 39), (('не', 'быть'), 37), (('и', 'я'), 34), (('потому', 'что'), 34), (('она', 'и'), 30), (('и', 'не'), 30), (('это', 'быть'), 29), (('не', 'мочь'), 29), (('она', 'не'), 29), (('сказать', 'он'), 27), (('что', 'она'), 25), (('я', 'в'), 25), (('я', 'и'), 24), (('то', 'что'), 24), (('он', 'не'), 24), (('когда', 'я'), 24), (('что', 'ты'), 24), (('джо', 'белл'), 23), (('он', 'и'), 23), (('что', 'он'), 22), (('у', 'она'), 22), (('она', 'быть'), 21)]
[(('сказать', 'она', 'и'), 11), (('о', 'д', 'берман'), 11), (('так', 'и', 'не'), 7), (('в', 'конец', 'конец'), 7), (('что', 'я', 'не'), 7), (('сказать', 'он', 'и'), 6), (('с', 'тот', 'пора'), 6), (('к', 'то', 'же'), 5), (('один', 'из', 'они'), 5), (('бар', 'джо', 'белла'), 5), (('не', 'знать', 'что'), 5), (('я', 'за', 'рука'), 5), (('мадам', 'сапфия', 'спанелла'), 5), (('то', 'что', 'я'), 5), (('не', 'мочь', 'быть'), 5), (('я', 'не', 'мочь'), 5), (('в', 'то', 'что'), 5

As we can see practically all bigrams are deprived of sense on their own but trigrams look more interesting: for example one of the most common trigrams is (**в**, **конец**, **конец**) which clearly refers to **в конце концов** -- it says a lot a bot the text guess. Another popular trigram is бар Джо Белла -- this one speaks for itself too.

---

The last task is as follows:
**Take 3-8 sentences from the original text and substitute some morphological information, for example, change the tense of the verbs, the number of the nouns, e.g, the original Слон подарил мартышке цветы should become Слоны подарят мартышкам цветок.**

In [ ]:
extra = 'Однажды поздно ночью я проснулся от  того,  что  мистер  Юниоши  что-то' +\
'кричал в пролет лестницы. Он жил на верхнем этаже, и голос  его,  строгий  и' +\
'сердитый, разносился по всему дому:' +\
'- Мисс Голайтли! Я должен протестовать!' +\
'Ребячливый, беззаботно-веселый голос ответил снизу:' +\
'- Миленький, простите! Я опять потеряла этот дурацкий ключ.'